# Introduction
This calculates all the linear intervals in each part in a given piece. Intervals are not counted if a rest between two notes is equal to or greater than two seconds.

In [ ]:
from music21 import *
import statistics
import numpy as np
import pandas as pd

In [ ]:
linear_corpus = corpus.corpora.LocalCorpus('DPhilCorpus_Linear_Comprehensive')
linear_corpus.existsInSettings
linear_corpus.addPath('~/Desktop/Oxford/Corpus/Sounding Corpus (Movements) Linear/Comprehensive/XML/')
linear_corpus.directoryPaths
linear_corpus.save()
linear_corpus.existsInSettings

In [3]:
pieceName = []
listOfWorks = linear_corpus.search('Op.')
for el in listOfWorks:
    pieceName.append(el.sourcePath)

In [4]:
#for a note that is tied to subsequent notes, this adds the duration of the later notes onto the first note, and then removes the later notes from the list of notes (i.e. it removes ties)
corpus_complete_list_of_parts = []
for el in pieceName:
    thePiece = corpus.parse(el)
    fileTitle = str(el).replace("/Users/joshua ballance/Desktop/Oxford/Corpus/Sounding Corpus (Movements) Linear/Comprehensive/XML/", "").replace(".xml", "").replace(".mxl", "")
    opusNumber = fileTitle.replace("Op.", "").replace(" ", "").replace("i", "").replace("v", "")
    movementNumber = fileTitle.replace("Op.", "").replace(" ", "").replace("1", "").replace("2", "").replace("3", "").replace("4", "").replace("5", "").replace("6", "").replace("7", "").replace("8", "").replace("9", "").replace("0", "")
    piece_complete_list_of_parts = [fileTitle]
    first_part = thePiece.parts[0]
    list_of_tempos = []
    stream_of_tempos = first_part.flat.getElementsByClass(tempo.MetronomeMark)
    for el in stream_of_tempos:
        list_of_tempos.append([el.offset, el])
    for h in range(0, len(thePiece.parts)):
        current_part = thePiece.parts[h]
        if h == 0:
            pass
        else:
            for el in list_of_tempos:
                current_part.insert(int(el[0]), el[1])
        stream_of_notes = current_part.flat.getElementsByClass(note.Note)
        listOfNotes = []
        for g in stream_of_notes:
            listOfNotes.append(g)
        listOfRests = current_part.flat.getElementsByClass(note.Rest)
        to_be_removed = []
        new_list_of_notes = []
        for i in range(0, len(listOfNotes)-1):
            first_note = listOfNotes[i]
            if 'start' in str(first_note.tie):
                subsequent_notes = listOfNotes[i+1:]
                list_of_tie_stops = []
                for j in range(i+1, len(listOfNotes)):
                    if 'stop' in str(listOfNotes[j].tie):
                        list_of_tie_stops.append(j)
                if len(list_of_tie_stops) == 0:
                    pass
                else:
                    last_note_position = min(list_of_tie_stops)
                    last_note = listOfNotes[last_note_position]
                    middle_note_positions = []
                    middle_notes = []
                    for k in range(0, len(listOfNotes)):
                        if k > i:
                            if k < last_note_position:
                                middle_note_positions.append(k)
                                middle_notes.append(listOfNotes[k])
                    quarter_lengths = []
                    seconds_lengths = []
                    quarter_lengths.append(first_note.quarterLength)
                    seconds_lengths.append(first_note.seconds)
                    quarter_lengths.append(last_note.quarterLength)
                    seconds_lengths.append(last_note.seconds)
                    if len(middle_notes) > 0:
                        for l in middle_notes:
                            if 'continue' in str(l.tie):
                                quarter_lengths.append(l.quarterLength)
                                seconds_lengths.append(l.seconds)
                    total_quarter_lengths = sum(quarter_lengths)
                    total_seconds_lengths = sum(seconds_lengths)
                    new_list_of_notes.append([listOfNotes[i], total_seconds_lengths, listOfNotes[i].offset])
                    for m in middle_note_positions:
                        to_be_removed.append(m)
                    to_be_removed.append(last_note_position)
            else:
                new_list_of_notes.append([listOfNotes[i],listOfNotes[i].seconds, listOfNotes[i].offset])
        if len(listOfNotes) > 1:
            final_note_position = len(listOfNotes)-1
            new_list_of_notes.append([listOfNotes[final_note_position], listOfNotes[final_note_position].seconds, listOfNotes[final_note_position].offset])
        for o in new_list_of_notes:
            if o[1] == 0:
                to_be_removed.append(new_list_of_notes.index(o))
        to_be_removed = list(dict.fromkeys(to_be_removed))
        to_be_removed.sort(reverse=True)
        new_new_list_of_notes = []
        for n in new_list_of_notes:
            if new_list_of_notes.index(n) in to_be_removed:
                pass
            else:
                new_new_list_of_notes.append(n)
        new_list_of_notes_and_rests = []
        rests_to_delete = []
        for m in range(0,len(listOfRests)):
            for n in listOfNotes:
                if n.seconds == 0:
                    pass
                else:
                    start_position = n.offset
                    end_position = n.offset+n.quarterLength
                    if listOfRests[m].offset >= start_position:
                        if listOfRests[m].offset < end_position:
                            rests_to_delete.append(m)
        for k in range(0, len(listOfRests)):    
            if k in rests_to_delete:
                pass
            else:
                new_list_of_notes_and_rests.append([listOfRests[k], listOfRests[k].seconds, listOfRests[k].offset])
        for l in new_new_list_of_notes:
            new_list_of_notes_and_rests.append(l)
        new_list_of_notes_and_rests = sorted(new_list_of_notes_and_rests, key = lambda x: x[2])
        part_info = [h]
        piece_complete_list_of_parts.append([part_info, new_list_of_notes_and_rests])
    corpus_complete_list_of_parts.append(piece_complete_list_of_parts)

In [5]:
max_rest_length = 2 #this is the maximum permitted duration for an intra-unit rest
list_of_intervals_absolute = []
for el in corpus_complete_list_of_parts:
    this_movement_intervals_absolute = []
    movement_title = el[0]
    parts = el[1:]
    for i in parts:
        intervals_list = []
        part_number = i[0]
        notes_and_rests = i[1]
        number_of_items = len(notes_and_rests)
        for j in range(0, len(notes_and_rests)-1):
            this_item = notes_and_rests[j]
            next_item = notes_and_rests[j+1]
            if 'note.Rest' in str(this_item):
                pass
            else:
                if 'note.Rest' in str(next_item):
                    subsequent_notes = []
                    subsequent_positions = []
                    for k in range(j+1, len(notes_and_rests)):
                        item_to_check = notes_and_rests[k]
                        if 'note.Rest' in str(item_to_check):
                            pass
                        else:
                            subsequent_notes.append(item_to_check)
                            subsequent_positions.append(k)
                    if len(subsequent_notes)>0:
                        next_note = subsequent_notes[0]
                        next_note_position = subsequent_positions[0]
                        intervening_rests = []
                        for l in range(j+1, next_note_position):
                            intervening_rests.append(notes_and_rests[l])
                        intervening_rest_durations = []
                        for m in intervening_rests:
                            intervening_rest_durations.append(m[1])
                        length_of_intervening_rest = sum(intervening_rest_durations)
                        if length_of_intervening_rest > max_rest_length:
                            pass
                        else:
                            firstNote = this_item[0]
                            secondNote = next_note[0]
                            intervalName = interval.Interval(firstNote, secondNote)
                            numberOfSemitones = abs(intervalName.semitones)
                            intervals_list.append(numberOfSemitones)
                else:
                    firstNote = this_item[0]
                    secondNote = next_item[0]
                    intervalName = interval.Interval(firstNote, secondNote)
                    numberOfSemitones = abs(intervalName.semitones)
                    intervals_list.append(numberOfSemitones)
        intervals_set = set(intervals_list)
        c = Counter(intervals_list)
        listOfCounts = []
        for l in intervals_set:
            info = [l, c[l]]
            listOfCounts.append(info)    
        if len(listOfCounts) == 0:
            pass
        else:
            this_movement_intervals_absolute.append(listOfCounts)
    list_of_intervals_absolute.append([movement_title, this_movement_intervals_absolute])

In [6]:
list_of_titles = ['Interval']
all_intervals = []
for i in range(0, len(list_of_intervals_absolute)):
    this_movement_info = list_of_intervals_absolute[i]
    movement_title = list_of_intervals_absolute[i][0]
    list_of_titles.append(movement_title)
    this_movement_intervals = this_movement_info[1:]
    for j in this_movement_intervals:
        for k in j:
            for l in k:
                all_intervals.append(l[0])
largest_interval = max(all_intervals)

In [ ]:
all_movements_all_interval_counts = []
just_intervals = []
for h in range(0, largest_interval+1):
    just_intervals.append(h)
all_movements_all_interval_counts.append(just_intervals)
for i in range(0, len(list_of_intervals_absolute)):
    this_movement_all_intervals = []
    this_movement_info = list_of_intervals_absolute[i]
    this_movement_intervals = this_movement_info[1:]
    for j in this_movement_intervals:
        for k in j:
            for l in k:
                this_movement_all_intervals.append(l)
    this_movement_all_interval_counts = []
    for m in range(0, largest_interval+1):
        this_interval_counts = []
        for n in this_movement_all_intervals:
            if n[0] == m:
                this_interval_counts.append(n[1])
        if len(this_interval_counts) == 0:
            this_movement_all_interval_counts.append(0)
        else:
            this_interval_total_count = sum(this_interval_counts)
            this_movement_all_interval_counts.append(this_interval_total_count)
    all_movements_all_interval_counts.append(this_movement_all_interval_counts)
df = pd.DataFrame(all_movements_all_interval_counts, index=list_of_titles)
df.columns = df.iloc[0]
df = df[1:]
df